#  **Gestión de propiedades y reservas de apartamentos en un portafolio**

## **Calendar** 

Muestra la representación detallada de la disponibilidad y el estado de las propiedades (apartamentos) en un determinado período de tiempo. Aquí está una explicación más detallada de cada campo en el bloque "Calendar":

1. __PropertyId (Id de la propiedad)__: Es una identificación única asociada a cada propiedad, que probablemente se refiere a un apartamento específico dentro del portafolio.

2. __CalendarDate (Fecha en el calendario)__: Representa una fecha específica entre el 01/07/2022 y el 31/12/2022.

3. __Blocked (Estado)__:
    * 0 (No reservado): Indica que el apartamento estaba disponible en esa fecha.
    * 1 (Bloqueado): Significa que el apartamento estaba cerrado y no estaba disponible para reservar en esa fecha.
    * 2 (Reservado): Indica que el apartamento estaba ocupado por clientes en esa fecha.

4. __Property_BookingId (Id de reserva de la propiedad)__: Este campo solo está presente si el estado (Blocked) es igual a 2, lo que significa que la propiedad estaba reservada en esa fecha. En este caso, el __Property_BookingId__ sería el identificador único asociado a esa reserva específica.

## **Propiedades (Properties)**
Proporciona información estática sobre las propiedades del portafolio, como su identificación única y la capacidad de personas que pueden alojarse en cada apartamento.

1. __PropertyId (Id de la propiedad)__: Es una identificación única asociada a cada propiedad (apartamento) en el portafolio.

2. __PR_Sleeps (Capacidad para dormir)__: Indica la capacidad del apartamento en personas, es decir, cuántas personas pueden dormir en ese apartamento.

## **Reservas (Bookings)**
Proporciona información detallada sobre las reservas realizadas, incluyendo el costo total, duración, fecha de creación y el canal a través del cual se realizó la reserva.

1. __Property_BookingId (Id de reserva de la propiedad)__: Identificación única asociada a una reserva específica.

2. __PB_TotalPrice (Precio total de la reserva)__: Representa el costo total de la reserva.

3. __PB_NumNights (Número de noches de la reserva)__: Indica cuántas noches está programada la reserva.

4. __PB_BookingCreatedDate (Fecha de creación de la reserva)__: Muestra la fecha en que se creó la reserva.

5. __PB_BookingChannelOriginId (Id del canal de origen de la reserva)__: Identificación del canal a través del cual se realizó la reserva.

In [75]:
# librerias
import pandas as pd
from herramientas import obtener_resumen_valores_unicos

In [76]:
# Especifica la ruta del archivo Excel
archivo_excel = 'data/Data analysis test.xlsx'

# Carga el archivo Excel en un DataFrame de pandas
calendar    = pd.read_excel(archivo_excel, sheet_name= 'Calendar')
Properties  = pd.read_excel(archivo_excel, sheet_name= 'Properties')
Bookings    = pd.read_excel(archivo_excel, sheet_name= 'Bookings')


## Visión general de la ocupación mensual 

* Calcula la ocupación mensual para cada propiedad en el portafolio. Utiliza los datos del bloque "Calendar" para contar las noches bloqueadas y las noches reservadas en cada mes.
* La ocupación sera igual : 

         Ocupación (%) = (Dias Reservados / Dias Disponibles) * 100 = (Dias Reservados / ((Fecha maxina - Fecha minima) - Noches Bloqueadad)) * 100

Con el fin de abordar esta situación, utilizaremos los datos de 'Calendar'. Al observar la tabla, se nota la ausencia de valores nulos, a excepción de 'Property_BookingId', que puede presentar un valor NaN cuando 'Blocked' es igual a 1.

In [77]:
obtener_resumen_valores_unicos(calendar)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos,Porcentaje Nulos
0,PropertyId,int64,223,38893,0,0.000000
1,CalendarDate,datetime64[ns],184,38893,0,0.000000
2,Blocked,int64,3,38893,0,0.000000
3,Property_BookingId,float64,6160,31168,7725,19.862186


Los datos comprenden el período desde el 31 de diciembre de 2022 a las 00:00:00 hasta el 1 de julio de 2022 a las 00:00:00, lo que equivale a un total de 184 días.

In [78]:
# Suponiendo que 'CalendarDate' es una columna de tipo datetime en tu DataFrame 'calendar'
calendar['CalendarDate'] = pd.to_datetime(calendar['CalendarDate'])

# Encuentra la fecha máxima y mínima
fecha_maxima = calendar['CalendarDate'].max()
fecha_minima = calendar['CalendarDate'].min()

# Calcula la diferencia en días
diferencia_en_dias = (fecha_maxima - fecha_minima).days
# print(f"Entre({fecha_maxima}) y ({fecha_minima}) hay un total de {diferencia_en_dias + 1} días")

Podemos utilizar **'PropertyId'** y, a través del indicador **'Blocked'**, calcular el número de días que la propiedad ha pasado en cada uno de los tres estados.

In [79]:
# Convertir la columna 'CalendarDate' a objetos de fecha
calendar['CalendarDate'] = pd.to_datetime(calendar['CalendarDate'])

# Resumen según los valores únicos en 'PropertyId'
resumen = calendar.groupby('PropertyId')['Blocked'].value_counts().unstack(fill_value=0)

# Suma de dias
resumen['Suma'] = resumen[0] + resumen[1]+ resumen[2]

# Imprimir el resumen
propiedades = len(resumen)
print(f"Se obtiene un total de {propiedades} propiedades diferentes, junto con la cantidad de días que han pasado en cada estado, así como la 'Suma'" )
resumen.head(15)


Se obtiene un total de 223 propiedades diferentes, junto con la cantidad de días que han pasado en cada estado, así como la 'Suma'


Blocked,0,1,2,Suma
PropertyId,,,,
2883,23,1,160,184
3963,35,1,148,184
3964,17,1,166,184
4138,21,7,156,184
4181,22,1,161,184
4259,31,2,151,184
4710,12,0,172,184
4785,19,1,164,184
4786,16,3,165,184


Al aplicar un filtro, podemos identificar 29 propiedades con una suma de días que difiere de 184. Esto podría deberse a la falta de registros para estas propiedades o a que ingresaron o quedaron fuera durante el período en el que se recopilaron los datos.

In [88]:
# Filtrar las filas donde 'Suma' es diferente de 184
df_filtrado = resumen.loc[resumen['Suma'] != 184]
df_ordenado = df_filtrado.sort_values(by='Suma', ascending=False)
df_ordenado

Blocked,0,1,2,Suma
PropertyId,,,,
43330,44,1,136,181
43332,16,3,154,173
43333,23,1,149,173
43334,19,2,150,171
43313,18,5,143,166
43337,24,2,138,164
43340,31,2,124,157
43338,45,11,97,153
43344,60,7,70,137


También es posible considerar la fecha máxima y la fecha mínima para cada propiedad, y a partir de estas fechas, calcular el número de días disponibles para cada propiedad. Si hay concordancia en los registros, es muy probable que algunas propiedades hayan quedado fuera o ingresado durante el período en el que se recopilaron los datos.

In [102]:
# Convertir la columna 'CalendarDate' al tipo de dato datetime
calendar['CalendarDate'] = pd.to_datetime(calendar['CalendarDate'])

# Agregar la nueva columna con el valor máximo de 'CalendarDate' para cada 'PropertyId'
calendar['MaxCalendarDate'] = calendar.groupby('PropertyId')['CalendarDate'].transform('max')
calendar['MinCalendarDate'] = calendar.groupby('PropertyId')['CalendarDate'].transform('min')

# Restar las columnas y crear una nueva columna 'DiasDiferencia'
calendar['DiasDiferencia'] = (calendar['MaxCalendarDate'] - calendar['MinCalendarDate']).dt.days
calendar['DiasDiferencia'] = calendar['DiasDiferencia'] + 1
calendar.head(15)

,PropertyId,CalendarDate,Blocked,Property_BookingId,MaxCalendarDate,MinCalendarDate,DiasDiferencia
0,42997,2022-07-01,2,103656.0,2022-12-31,2022-07-01,184
1,42997,2022-07-02,2,103656.0,2022-12-31,2022-07-01,184
2,42997,2022-07-03,0,NaN,2022-12-31,2022-07-01,184
3,42997,2022-07-04,2,102879.0,2022-12-31,2022-07-01,184
4,42997,2022-07-05,2,102879.0,2022-12-31,2022-07-01,184
5,42997,2022-07-06,2,102879.0,2022-12-31,2022-07-01,184
6,42997,2022-07-07,2,102879.0,2022-12-31,2022-07-01,184
7,42997,2022-07-08,2,102879.0,2022-12-31,2022-07-01,184
8,42997,2022-07-09,2,102879.0,2022-12-31,2022-07-01,184
9,42997,2022-07-10,2,102879.0,2022-12-31,2022-07-01,184


Filtramos los datos para cuando 'DiasDiferencia' es diferente a 184 para comparar con los datos anteriores

In [95]:
# Imprimir el DataFrame resultante
df_filtrado = calendar.loc[calendar['DiasDiferencia'] != 184]
df_filtrado = df_filtrado[['PropertyId', 'DiasDiferencia']]
df_filtrado= df_filtrado.drop_duplicates()
df_ordenado = df_filtrado.sort_values(by='DiasDiferencia', ascending=False)
df_ordenado

,PropertyId,DiasDiferencia
35403,43330,181
35584,43332,173
35757,43333,173
35930,43334,171
34317,43313,166
36101,43337,164
36532,43340,157
36265,43338,153
36876,43344,137
37287,43347,137


Las tablas coinciden; podemos asumir que algunas propiedades quedaron fuera o ingresaron durante el período en el que se recopilaron los datos. Con esta información, podemos calcular el porcentaje de ocupación para cada propiedad tomando en cuenta el tiempo que la propiedad formo parte del portafolio.

            Ocupación (%) = (Dias Reservados / Dias Disponibles) * 100

In [101]:
resumen['ocupacion(%)'] = (resumen[2] / (resumen['Suma'] - resumen[1])) * 100
resumen.head(15)

Blocked,0,1,2,Suma,ocupacion(%)
PropertyId,,,,,
2883,23,1,160,184,87.431694
3963,35,1,148,184,80.874317
3964,17,1,166,184,90.710383
4138,21,7,156,184,88.135593
4181,22,1,161,184,87.978142
4259,31,2,151,184,82.967033
4710,12,0,172,184,93.478261
4785,19,1,164,184,89.617486
4786,16,3,165,184,91.160221


In [103]:
df_filtrado = resumen.loc[resumen['Suma'] != 184]
df_filtrado.head(15)

Blocked,0,1,2,Suma,ocupacion(%)
PropertyId,,,,,
43097,8,0,83,91,91.208791
43098,8,0,83,91,91.208791
43100,11,0,80,91,87.912088
43101,4,0,87,91,95.604396
43313,18,5,143,166,88.819876
43330,44,1,136,181,75.555556
43332,16,3,154,173,90.588235
43333,23,1,149,173,86.627907
43334,19,2,150,171,88.757396


In [ ]:
# Filtrar el DataFrame cuando 'PropertyId' es igual a 22
Calendar_filtrado = calendar.loc[calendar['PropertyId'] == 43345]

# Imprimir el DataFrame resultante después del filtro
Calendar_filtrado

,PropertyId,CalendarDate,Blocked,Property_BookingId,MaxCalendarDate,MinCalendarDate,DiasDiferencia
37013,43345,2022-08-17,0,NaN,2022-12-31,2022-08-17,137
37014,43345,2022-08-18,0,NaN,2022-12-31,2022-08-17,137
37015,43345,2022-08-19,0,NaN,2022-12-31,2022-08-17,137
37016,43345,2022-08-20,0,NaN,2022-12-31,2022-08-17,137
37017,43345,2022-08-21,0,NaN,2022-12-31,2022-08-17,137
...,...,...,...,...,...,...,...
37145,43345,2022-12-27,0,NaN,2022-12-31,2022-08-17,137
37146,43345,2022-12-28,0,NaN,2022-12-31,2022-08-17,137
37147,43345,2022-12-29,0,NaN,2022-12-31,2022-08-17,137
37148,43345,2022-12-30,0,NaN,2022-12-31,2022-08-17,137
